In [75]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import warnings
warnings.filterwarnings('ignore')
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
from datetime import datetime, timedelta
import os
chromedriver = "/Users/stevenwang/Projects/funded-and-hiring-site/newsletter_scraper/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
import pyperclip
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pytest_timeout

In [76]:
def get_ttnews():
    url = 'https://www.ttnews.com/articles'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    # news = []
    companies = []
    deal_text = []
    publish_date = []
    company_urls = []
    stories = []
    
    articles = soup.find_all(class_='views-row')
    # print(articles)
    for article in articles:
        # print(article)
        title = article.find('a')
        date =  article.find(class_='date-display-single')
        text =  article.find('p')

        companies.append('TTNews')
        deal_text.append(text)
        publish_date.append(date)
        stories.append(title.get('href'))
        company_urls.append('https://ttnews.com/')
        
        # print(title)
        # print(title.get('href'))
        # print(date.get_text())
        # print(text.get_text())
        # , date.get_text(), text.get_text())

        # print(title)
        # news.append(News(title.get_text(), title.get('href')))

    dataframe = pd.DataFrame({'company':companies, 'deal_text':deal_text, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls})
    return dataframe

In [77]:
ttnews = get_ttnews()

In [78]:
def get_axios_deals(days_back):
    url = 'https://www.axios.com/authors/danprimack/newsletters'
#     url = 'https://www.axios.com/authors/kiakokalitcheva/newsletters'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    companies = []
    deal_text = []
    publish_date = []
    stories = []
    company_urls = []

    #grab last 5 edition urls, check to make sure they are all within the last week
    news = soup.find_all('amp-layout')[:days_back]
    for letter in news:
        #check if date of newsletter is in last week, if not then pass
        date = letter.find_next('span', attrs = {'data-testid':'time-rubric'}).text
        if 'hours' in date or 'mins' in date or 'hour' in date:
            date = pd.to_datetime(datetime.now().date())
        else:
            date = pd.to_datetime(date)
        if date < pd.to_datetime(datetime.now().date()) - timedelta(days = days_back):
            break

        #get company names and deal texts mentioned in each news letter
        letter_url = letter.find_next('a', href = True, class_ = 'title-link gtm-content-click')['href']
        letter_companies, letter_deal_text, letter_story_urls = get_axios_deal_text(letter_url)

        #append these to the running list
        companies += letter_companies
        deal_text += letter_deal_text
        publish_date += [date for i in range(len(letter_companies))]
        stories += letter_story_urls
    
    #grab company site urls
    for company, story in zip(companies, stories):
        company = company.lower().strip()
        if company in story: #if company name is in story url, then use story url as company url
            company_urls.append(story)
        else:#grab company url from story
            try:
                company_site = get_company_url_from_axios_story(company, story)
                company_urls.append(company_site)
            except: #if company url can't be found in the story, use story url instead
                company_urls.append(story)
                
    dataframe = pd.DataFrame({'company':companies, 'deal_text':deal_text, 'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls})
    return dataframe

def get_axios_deal_text(link):
    prorata_soup =  BeautifulSoup(requests.get(link).content, "html.parser")
    vc_deals = prorata_soup.find('div', text = re.compile('Venture Capital Deals')).find_next('div', class_ = re.compile('.*story-text.*')).find_all('p')
    
    companies = []
    deal_text = []
    stories = []

    for deal in vc_deals:
        company_name = deal.find_next('strong').text.strip()
        if company_name == '•':
            company_name = deal.find_next('strong').find_next('strong').text.strip()

        text = deal.text
        text = re.sub(r'[^-a-zA-z0-9.,!? ]', '', text).strip()
        story_url = deal.find_next('a', class_ = 'gtm-content-click')['href']

        companies.append(company_name)
        deal_text.append(text)
        stories.append(story_url)
    return(companies, deal_text, stories)

def get_company_url_from_axios_story(company_name, story_url):
    page = requests.get(story_url)
    soup = BeautifulSoup(page.content, "html.parser")
    article_url = soup.find('meta', attrs={'property': 'twitter:url'})['content']
    headers = {'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'}
    article_page = requests.get(article_url, headers = headers)
    article_soup = BeautifulSoup(article_page.content, "html.parser") 

    potential_company_urls = []
    #get hyperlink url that contains company name
    company_name_no_space = company_name.replace(' ', '')
    company_name_no_space_url = article_soup.find_all('a', href = re.compile('.*{company_name_no_space}'.format(company_name_no_space = company_name_no_space), re.IGNORECASE))
    if len(company_name_no_space_url) > 0:
        potential_company_urls.append(company_name_no_space_url[0]['href'])
    
    company_name_hyphen = company_name.replace(' ', '-')
    company_name_hyphen_url = article_soup.find_all('a', href = re.compile('.*{company_name_hyphen}'.format(company_name_hyphen = company_name_hyphen), re.IGNORECASE))
    if len(company_name_hyphen_url) > 0:
        potential_company_urls.append(company_name_hyphen_url[0]['href'])
        
    company_name_url = article_soup.find_all('a', text = re.compile('.*{company_name}.*'.format(company_name = company_name), re.IGNORECASE))
    if len(company_name_url)  > 0:
        potential_company_urls.append(company_name_url[0]['href'])
    
    #return the shortest url
    return min(potential_company_urls, key=len)

In [79]:
def clean_axios_deals(deals):
    #get deal size
    deals['deal_size'] = deals.deal_text.str.extract(r'((?:\d+[.]\d+|\d+|\d+[.]\d+ |\d+ )(?:million|billion|Million|Billion))')
    
    #only keep deals that have dollar amounts
    deals = deals[deals.deal_size.notnull()]
    
    #convert to an actual number
    deals.loc[deals.deal_size.str.contains('million|m$', case = False), 'deal_size'] = deals.deal_size.str.split(' ').str[0].astype(float) * 1e6
    deals['deal_size'] = deals.deal_size.astype(str)
    deals.loc[deals.deal_size.str.contains('billion|b$', case = False), 'deal_size'] = deals.deal_size.str.split(' ').str[0].astype(float) * 1e9
    
    #remove urls from deal text
    deals['deal_text'] = deals.deal_text.str.split('www[.]|http').str[0].str.strip()
    return deals

In [80]:
# axios = get_axios_deals(7)

# axios

# axios = clean_axios_deals(axios)

In [81]:
def get_finsme_deals(days_back):
    page = 1
    past_date = False

    companies = []
    deal_text = []
    publish_date = []
    stories = []
    company_urls = []
    while page <= days_back and past_date == False:
        url = 'http://www.finsmes.com/category/usa/page/' + str(page)
        soup = BeautifulSoup(requests.get(url).content, "html.parser")
        deals = soup.find_all('article')
        for d in deals:
            #check if date of article is > 7 days ago, if true stop
            date = pd.to_datetime(d.find_next('time', attrs = {'itemprop':'datePublished'}).text)
            if date < pd.to_datetime(datetime.now().date()) - timedelta(days = days_back):
                past_date = True
                break
            else:
                text = d.find_next('p').text.strip().replace('\n', '')
                company_name = text.split(',')[0].strip()
                story_url = d.find('a', attrs = {'itemprop':'url'})['href']

                companies.append(company_name)
                deal_text.append(text)
                publish_date.append(date)
                stories.append(story_url)

        page += 1
    #grab company site urls
    for company, story in zip(companies, stories):
        company_site = get_company_url_from_finsmes_story(company, story)
        company_urls.append(company_site)

    dataframe = pd.DataFrame({'company':companies, 'deal_text':deal_text, 
                              'publish_date':publish_date, 'story_url':stories, 'company_url':company_urls})
    return dataframe

def clean_finsme_deals(deals):
    #remove acquisitions
    deals = deals[(~deals.company.str.contains('acquire|acquisition')) & (~deals.deal_text.str.contains('acquire|acquisition'))]
    
    #remove deals where company name is hard to parse
    deals = deals[deals.company.str.split(' ').str.len() <= 5]
    
    #get deal size
    deals['deal_size'] = deals.deal_text.str.extract(r'((?:\d+[.]\d+|\d+)(?:million|billion|m|b|M))')
    deals = deals[deals.deal_size.notnull()]
    
    deals.loc[deals.deal_size.str.contains('million|m$', case = False), 'deal_size'] = deals.deal_size.str.split('m|M').str[0].astype(float) * 1e6
    deals['deal_size'] = deals.deal_size.astype(str)
    deals.loc[deals.deal_size.str.contains('billion|b$', case = False), 'deal_size'] = deals.deal_size.str.split('b|B').str[0].astype(float) * 1e9
    
    
    return deals

def get_company_url_from_finsmes_story(company_name, story_url):
    headers = {'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'}
    article_page = requests.get(story_url, headers = headers)
    article_soup = BeautifulSoup(article_page.content, "html.parser") 
    article_content = article_soup.find('div', class_ = 'entry-content')
    try:
        company_url = article_content.find_all('a', text = re.compile('.*{company_name}.*'.format(company_name = company_name), re.IGNORECASE))[0]['href']
    except:
        company_url = story_url
    return company_url

In [82]:
# finsme = get_finsme_deals(7)
# finsme = clean_finsme_deals(finsme)

In [83]:
vc_deals = pd.concat([ttnews], axis = 0)
#replace all .coms in company names
vc_deals['company'] = vc_deals.company.str.replace("[.].*|•|’|,|[+]|[()]|[#]", '').str.strip()

#remove unicode spaces
# vc_deals['company'] = vc_deals.company.str.replace(u'\xa0|\u200d', ' ').str.strip()
# vc_deals['company'] = vc_deals.company.str.replace('&', 'and').str.strip()
# vc_deals['company'] = vc_deals.company.str.replace('[/]', '').str.strip()

# vc_deals.loc[vc_deals.company == '🧀', 'company'] = 'Nobell Foods'
# vc_deals['company'] = vc_deals.company.str.replace('☕', '')
# vc_deals['company'] = vc_deals.company.str.replace('🚙', '')
# vc_deals['company'] = vc_deals.company.str.replace('🚑', '')
# vc_deals['company'] = vc_deals.company.str.replace('🐶', '')
# vc_deals['company'] = vc_deals.company.str.replace('⛽️|🐉', '')


# vc_deals['company'] = vc_deals.company.str.strip()
# vc_deals = vc_deals[vc_deals.company != '']
# vc_deals = vc_deals[vc_deals.company != 'Vestiaire Collective']



# vc_deals = vc_deals[~vc_deals.company.str.upper().duplicated()]
# vc_deals['deal_size'] = vc_deals.deal_size.astype(float)
# vc_deals = vc_deals.sort_values('deal_size', ascending = False)

In [84]:
len(vc_deals)

25

In [85]:
vc_deals

,company,deal_text,publish_date,story_url,company_url
0,TTNews,"[ Lordstown Motors Corp., the electric-truck m...","[September 30, 2021]",/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/
1,TTNews,[ ABB Ltd. is preparing to roll out the first ...,"[September 30, 2021]",/articles/new-ev-charger-can-provide-62-miles-...,https://ttnews.com/
2,TTNews,[ After sounding an optimistic note in the fir...,"[September 30, 2021]",/articles/car-sales-drop-chip-shortages-stymie...,https://ttnews.com/
3,TTNews,[ WASHINGTON — His government overhaul plans a...,"[September 29, 2021]",/articles/progress-budget-talks-will-deal-beat...,https://ttnews.com/
4,TTNews,[ The outlook for transportation and logistics...,"[September 29, 2021]",/articles/what-q4-and-beyond-have-store-transp...,https://ttnews.com/
5,TTNews,[ LONDON — The British government’s reserve ta...,"[September 29, 2021]",/articles/soldiers-will-haul-fuel-ease-uk-crisis,https://ttnews.com/
6,TTNews,[ Navistar Inc. announced the launch of the re...,"[September 29, 2021]",/articles/navistar-launches-diesel-version-lat...,https://ttnews.com/
7,TTNews,[ Legal experts and researchers say juries gen...,"[September 29, 2021]",/articles/negative-jury-attitudes-can-contribu...,https://ttnews.com/
8,TTNews,[ Walmart Inc. is expanding its hiring push wi...,"[September 29, 2021]",/articles/walmart-stokes-hiring-rush-plan-add-...,https://ttnews.com/
9,TTNews,"[ When the first snub-nosed, electric van roll...","[September 29, 2021]",/articles/electric-vans-roll-line-once-made-ga...,https://ttnews.com/


In [86]:
# def get_lever_jobs(companies):
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_lever = []

#     for company in companies:
#         company_cleaned = company.replace(' ', '').lower()
#         url = 'https://jobs.lever.co/' + company_cleaned
#         soup = BeautifulSoup(requests.get(url).content, 'html.parser')
#         jobs = soup.find_all('div', class_ = 'posting')

#         #if no jobs found, add to no lever list
#         if len(jobs) == 0:
#             no_lever.append(company)
#         else:
#             for job in jobs:
#                 job_name = job.find('h5', attrs = {'data-qa':'posting-name'}).text.strip()

#                 job_location = job.find('span', attrs = {'class':'sort-by-location posting-category small-category-label'})
#                 if job_location == None: #not all jobs have location, set to none if missing
#                     job_location = None
#                 else:
#                     job_location = job_location.text.strip()

#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
    
#     print(len(companies) - len(no_lever), 'out of ', len(companies), 'companies found on Lever')
#     return df, no_lever
    

In [87]:
# lever_jobs, not_on_lever = get_lever_jobs(vc_deals.company)

In [88]:
# import signal

# class Timeout:
#     def __init__(self, seconds=1, error_message='Timeout'):
#         self.seconds = seconds
#         self.error_message = error_message
#     def handle_timeout(self, signum, frame):
#         raise TimeoutError(self.error_message)
#     def __enter__(self):
#         signal.signal(signal.SIGALRM, self.handle_timeout)
#         signal.alarm(self.seconds)
#     def __exit__(self, type, value, traceback):
#         signal.alarm(0)

# def test_it_doesnt_succeed():
#     try:
#         with Timeout(seconds=6):
#             do_the_thing()
#     except TimeoutError:
#         pass
#     else:
#         raise AssertionError('Expected the thing to timeout!')

In [89]:
# company_ls = []
# job_ls = []
# location_ls = []
# listing_url = []
# no_greenhouse = []

# for company in companies:
#     company_cleaned = company.replace(' ', '')
#     url = 'https://boards.greenhouse.io/' + company_cleaned
#     print(url)
#     try:
# #         test_it_doesnt_succeed(requests.get(url).content, 'html.parser')
#         soup = BeautifulSoup(requests.get(url).content, 'html.parser')
#     except:
#         no_greenhouse.append(company)
#         continue
#     jobs = soup.find_all('div', class_ = 'opening')

#     #if no jobs found, add to no lever list
#     if len(jobs) == 0:
#         no_greenhouse.append(company)
#     else:
#         for job in jobs:
#             job_name = job.find('a', attrs = {'data-mapped':"true"}).text.strip()
#             job_location = job.find('span', class_ = 'location').text.strip()

#             company_ls.append(company)
#             job_ls.append(job_name)
#             location_ls.append(job_location)
#             listing_url.append(url)
# df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})

In [90]:
# def get_greenhouse_jobs(companies):
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_greenhouse = []

#     for company in companies:
#         company_cleaned = company.replace(' ', '')
#         url = 'https://boards.greenhouse.io/' + company_cleaned
#         try:
#             soup = BeautifulSoup(requests.get(url).content, 'html.parser')
#         except:
#             no_greenhouse.append(company)
#             continue
#         jobs = soup.find_all('div', class_ = 'opening')

#         #if no jobs found, add to no lever list
#         if len(jobs) == 0:
#             no_greenhouse.append(company)
#         else:
#             for job in jobs:
#                 job_name = job.find('a', attrs = {'data-mapped':"true"}).text.strip()
#                 job_location = job.find('span', class_ = 'location').text.strip()

#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_greenhouse), 'out of ', len(companies), 'companies found on greenhouse')
#     return(df, no_greenhouse)

In [91]:
# greenhouse_jobs, not_on_greenhouse = get_greenhouse_jobs(not_on_lever)

In [92]:
# def get_breezy_jobs(companies):
#     headers = {'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'}
    
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_breezy = []
    
#     for company in companies:
#         company_cleaned = company.replace(' ', '').lower() 
#         url = 'https://' + company_cleaned + '.breezy.hr'
#         soup = BeautifulSoup(requests.get(url, headers = headers).content, 'html.parser')
        
#         jobs = soup.find_all('li', attrs = {'class':'position transition'})
        
#         #if no jobs found, add to no breezy list
#         if len(jobs) == 0:
#             no_breezy.append(company)
#         else:
#             for job in jobs:
#                 job_name = job.find_next('h2').text.strip()
#                 job_location = job.find_next('li', attrs = {'class':'location'}).text.strip()
                
#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)
                
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_breezy), 'out of ', len(companies), 'companies found on breezy')
#     return df, no_breezy


In [93]:
# breezy_jobs, not_on_breezy = get_breezy_jobs(not_on_greenhouse)

In [94]:
# def get_google_jobs(companies):
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_google = []

#     for company in companies:
#         company_cleaned = company.replace(' ', '').lower()
#         url = 'https://hire.withgoogle.com/public/jobs/' + company_cleaned
#         soup = BeautifulSoup(requests.get(url).content, 'html.parser')
#         jobs = soup.find_all('li', class_ = 'bb-public-jobs-list__job-item ptor-jobs-list__item')

#         #if no jobs found, add to no lever list
#         if len(jobs) == 0:
#             no_google.append(company)
#         else:
#             for job in jobs:
#                 job_name = job.find('span', class_ = 'bb-public-jobs-list__job-item-title ptor-jobs-list__item-job-title').text.strip()
#                 job_location = job.find('span', class_ = 'bb-public-jobs-list__job-item-location ptor-jobs-list__item-location').text.strip()

#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_google), 'out of ', len(companies), 'companies found on hirewithgoogle')
#     return(df, no_google)

In [95]:
# google_jobs, not_on_google = get_google_jobs(not_on_greenhouse)

In [96]:
# def get_angel_jobs(companies):
# #     driver = webdriver.Chrome(chromedriver)
    
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_angel = []

#     for company in companies:
#         driver = webdriver.Chrome(chromedriver)
#         company_cleaned = company.replace(' ', '').lower()
#         url = 'https://angel.co/company/' + company_cleaned + '/jobs'
#         driver.get(url)
        
#         soup = BeautifulSoup(driver.page_source, "html.parser")
#         jobs = soup.find_all('div', class_ = 'styles_component__1_YxE styles_expanded__31zII')

#         #if no jobs found, add to no lever list
#         if len(jobs) == 0:
#             no_angel.append(company)
#         else:
#             for job in jobs:
#                 job_name = job.find('div', class_ = 'breakpoint__desktop-up').text.strip()
#                 job_location = job.find('div', class_ = 'styles_component__26gqE styles_truncate__dUufp styles_location__ACesY').text.strip()

#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)

#         driver.quit()
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_angel), 'out of ', len(companies), 'companies found on angel.co')
# #     driver.quit()
#     return(df, no_angel)

In [97]:
# angel_jobs, not_on_angel = get_angel_jobs(not_on_greenhouse)

In [98]:
# def get_smartrecruiter_jobs(companies):
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_smartrecruiter = []

#     for company in companies:
#         company_cleaned = company.replace(' ', '')
#         url = 'https://careers.smartrecruiters.com/' + company_cleaned
#         soup = BeautifulSoup(requests.get(url).content, 'html.parser')

#         locations = soup.find_all('section', class_ = 'openings-section opening opening--grouped js-group')
        
#         if len(locations) == 0:
#             no_smartrecruiter.append(company)
#         else:
#             for loc in locations:
#                 job_location = loc.find('h3', class_ = 'opening-title title display--inline-block text--default').text.strip()
#                 jobs = loc.find_all('h4', class_ = 'details-title job-title link--block-target')
#                 for job in jobs:
#                     company_ls.append(company)
#                     job_name = job.text.strip()
#                     job_ls.append(job_name)
#                     location_ls.append(job_location)
#                     listing_url.append(url)
                    
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_smartrecruiter), 'out of ', len(companies), 'companies found on smartrecruiter')
#     return(df, no_smartrecruiter)



In [99]:
# sr_jobs, not_on_sr = get_smartrecruiter_jobs(not_on_breezy)

In [100]:
# def get_workable_jobs(companies):
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_workable = []
    
#     for company in companies:
#         driver = webdriver.Chrome(chromedriver)
#         company_cleaned = company.replace(' ', '').lower()
#         url = 'https://apply.workable.com/' + company_cleaned

#         driver.get(url)
#         try:
#         #wait until job list shows up
#             WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//li[@data-ui='job-opening']")))
#         except:
#             no_workable.append(company)
#             driver.quit()
#             continue
#         soup = BeautifulSoup(driver.page_source, "html.parser")
        
#         #click load more jobs
#         load_more_button = soup.find_all(attrs = {'data-ui':'load-more-button'})
#         while len(load_more_button) > 0:
#             btn = driver.find_element_by_xpath("//button[@data-ui='load-more-button']")
#             btn.click()
#         #     try:
#         #         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[@data-ui='load-more-button']")))
#         #     except:
#         #         pass
#             time.sleep(2)
#             soup = BeautifulSoup(driver.page_source, "html.parser")
#             load_more_button = soup.find_all(attrs = {'data-ui':'load-more-button'})
        
#         jobs_list = soup.find_all('li', attrs = {'data-ui':'job-opening'})

#         if len(jobs_list) == 0:
#             no_workable.append(company)
#         else:
#             for job in jobs_list:
#                 job_name = job.find_next(attrs = {'data-id':'job-item'}).text
#                 job_url = 'https://apply.workable.com' + job.find_next('a')['href']
#                 try:
#                     job_location = job.find_next(attrs = {'data-ui':'job-location'}).text
#                 except:
#                     job_location = None
                
#                 #check if remote job
#                 if len(job.find_all(attrs = {'data-ui':'job-remote'})) > 0:
#                     job_location = 'Remote'

#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)
            
#         driver.quit()
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_workable), 'out of ', len(companies), 'companies found on workable')
#     return(df, no_workable)

In [101]:
# workable_jobs, not_on_workable = get_workable_jobs(not_on_sr)

In [102]:
# driver = webdriver.Chrome(chromedriver)

# url = 'https://apply.workable.com/zenoti'
# driver.get(url)

# #wait until job list shows up
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//li[@data-ui='job-opening']")))

# soup = BeautifulSoup(driver.page_source, "html.parser")

# #click load more jobs
# load_more_button = soup.find_all(attrs = {'data-ui':'load-more-button'})
# while len(load_more_button) > 0:
#     btn = driver.find_element_by_xpath("//button[@data-ui='load-more-button']")
#     btn.click()
# #     try:
# #         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//button[@data-ui='load-more-button']")))
# #     except:
# #         pass
#     time.sleep(2)
#     soup = BeautifulSoup(driver.page_source, "html.parser")
#     load_more_button = soup.find_all(attrs = {'data-ui':'load-more-button'})

# jobs_list = soup.find_all('li', attrs = {'data-ui':'job-opening'})

# for job in jobs_list:
#     job_name = job.find_next(attrs = {'data-id':'job-item'}).text
#     job_url = 'https://apply.workable.com' + job.find_next('a')['href']
    
#     job_location = job.find_next(attrs = {'data-ui':'job-location'}).text
#     #check if remote job
#     if len(job.find_all(attrs = {'data-ui':'job-remote'})) > 0:
#         job_location = 'Remote'
#     print(job_name, job_url, job_location)

In [103]:
# def get_indeed_jobs(companies):
#     company_ls = []
#     job_ls = []
#     location_ls = []
#     listing_url = []
#     no_indeed = []
#     headers = {'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'}


#     for company in companies:
#         company_cleaned = company.replace(' ', '-')
#         url = 'https://www.indeed.com/cmp/' + company_cleaned + '/jobs?q=&l=#cmp-skip-header-desktop'
#         soup = BeautifulSoup(requests.get(url, headers = headers).content, 'html.parser')
#         jobs = soup.find_all('li', class_ = 'cmp-JobListItem')

#         #if no jobs found, add to no lever list
#         if len(jobs) == 0:
#             no_indeed.append(company)
#         else:
#             for job in jobs:
#                 job_name = job.find('div', class_ = 'cmp-JobListItem-title').text.strip()
#                 job_location = job.find('div', class_ = 'cmp-JobListItem-subtitle').text.strip()

#                 company_ls.append(company)
#                 job_ls.append(job_name)
#                 location_ls.append(job_location)
#                 listing_url.append(url)
#     df = pd.DataFrame({'company':company_ls, 'job':job_ls, 'location':location_ls, 'job_page':listing_url})
#     print(len(companies) - len(no_indeed), 'out of ', len(companies), 'companies found on indeed')
#     return(df, no_indeed)

In [104]:
# indeed_jobs, not_on_indeed = get_indeed_jobs(not_on_workable)

In [105]:
#concat jobs
# jobs = pd.concat([lever_jobs, greenhouse_jobs, indeed_jobs, sr_jobs, workable_jobs], axis = 0)
# jobs = lever_jobs

In [106]:
# len(jobs.company.unique())

In [107]:
# jobs.job.unique().tolist()

In [108]:
#remote jobs
# jobs['Remote'] = 0
# jobs.loc[(jobs.location.str.contains('Remote', case = False, na = False)) | 
#          (jobs.job.str.contains('Remote', case = False, na = False)), 'Remote'] = 1

In [109]:
#job categories
# jobs['Engineering/Technology'] = 0
# eng_str = 'engineer|developer|quality assurance|Technical Lead|tech lead|devops|database admin|(machine|deep) learning|scientist|\Wqa\W|infrastructure|\WML\W|tech.*lead|architect|chemist|research|geneticist|computational|informatics|scientist|\WIT\W|programmer|\WML\W|biolog|pharma|oncology|chemical|dev op|computational|cloud|Bioinformatician'
# jobs.loc[jobs.job.str.contains(eng_str, case = False), 'Engineering/Technology'] = 1

# jobs['Data'] = 0
# data_str = 'database admin|(machine|deep) learning|scientist|data|analytics|\WML\W|business intelligence|BI Analyst|\WAI\W|product analyst|statistician|\WBI\W|DBA'
# jobs.loc[jobs.job.str.contains(data_str, case = False), 'Data'] = 1

# #group data into engineering/technology
# jobs.loc[jobs.Data == 1, 'Engineering/Technology'] = 1

# jobs['Design'] = 0
# design_str = 'design|user experience|\Wux\W|\Wui\W|creative director|\Wart\W|graphic'
# jobs.loc[jobs.job.str.contains(design_str, case = False), 'Design'] = 1

# jobs['Product'] = 0
# prod_str = 'product'
# jobs.loc[jobs.job.str.contains(prod_str, case = False), 'Product'] = 1

# jobs['Sales/Business Development/Growth'] = 0
# sales_str = 'sales|business development|demand generation|deal desk|growth|sdr|revenue|expansion'
# jobs.loc[jobs.job.str.contains(sales_str, case = False), 'Sales/Business Development/Growth'] = 1

# jobs['Operations'] = 0
# op_str = 'operations|program manager|project manager|chief of staff|strateg|general manager|scrum'
# jobs.loc[jobs.job.str.contains(op_str, case = False), 'Operations'] = 1

# jobs['Marketing'] = 0
# marketing_str = 'marketing|growth|SEO|market research'
# jobs.loc[jobs.job.str.contains(marketing_str, case = False), 'Marketing'] = 1

# jobs['Customer Success'] = 0
# cs_str = 'account (executive|manager)|customer success|customer (support|care|service)|implementation|Deployment Specialist|customer|client|crm|success|member experience|onboard'
# jobs.loc[jobs.job.str.contains(cs_str, case = False), 'Customer Success'] = 1

# jobs['Content Creation'] = 0
# content_str = 'content|copywrite|communications|\Wpr\W|write|public relation'
# jobs.loc[jobs.job.str.contains(content_str, case = False), 'Content Creation'] = 1

# jobs['HR/People'] = 0
# hr_str = '\Whr\W|talent|recruiter|sourcer|people|payroll|diversity|inclusion|compensation|benefits'
# jobs.loc[jobs.job.str.contains(hr_str, case = False), 'HR/People'] = 1

# jobs['Legal/Compliance'] = 0
# legal_str = 'legal|compliance|counsel|Regulatory|attorney|litigation|lawyer'
# jobs.loc[jobs.job.str.contains(legal_str, case = False), 'Legal/Compliance'] = 1

# jobs['Finance/Accounting'] = 0
# fin_str = 'finance|financial|accountant|accounting|comptroller|controller|payroll|audit|M&A|payable|corporate development|credit|investment|accounts receivable'
# jobs.loc[jobs.job.str.contains(fin_str, case = False), 'Finance/Accounting'] = 1

# jobs['Supply Chain/Logistics'] = 0
# scl_str = 'supply chain|logistics|warehouse|inventory|buyer|procure|purchasing|acquisition'
# jobs.loc[jobs.job.str.contains(scl_str, case = False), 'Supply Chain/Logistics'] = 1


# jobs['Other'] = 0
# jobs.loc[(jobs['Engineering/Technology'] == 0) & (jobs['Data'] == 0) & (jobs['Design'] == 0) & (jobs['Product'] == 0) & (jobs['Finance/Accounting'] == 0) & 
#          (jobs['Marketing'] == 0) & (jobs['Sales/Business Development/Growth'] == 0) & (jobs['Operations'] == 0) & (jobs['Customer Success'] == 0) & 
#          (jobs['Content Creation'] == 0) & (jobs['HR/People'] == 0) & (jobs['Legal/Compliance'] == 0) & (jobs['Supply Chain/Logistics'] == 0), 'Other'] = 1


In [110]:
# jobs = jobs[~jobs.company.isin(['Beacon', 'Canopy', 'DCM', 'harbor'])]

In [111]:
# len(jobs)

In [112]:
# jobs_grp = jobs.groupby(['company', 'job_page'])[[col for col in jobs.columns if col not in ['company', 'job', 'location', 'url']]].sum()
# jobs_grp = jobs_grp.reset_index()

In [113]:
# jobs_grp

In [114]:
# jobs_grp.loc[jobs_grp.company == 'vgs', 'company'] = 'Very Good Security'

In [115]:
# jobs_grp.loc[jobs_grp.company == 'joinclubhouse', 'company'] = 'Clubhouse'

In [116]:
# final = pd.merge(vc_deals, jobs_grp, on = 'company', how = 'inner', copy = False)

# final = pd.merge(vc_deals, [], on = 'company', how = 'inner', copy = False)
# final = final.reset_index(drop = True)

final = vc_deals

In [117]:
# final = final[final.company != 'DISCO']

In [118]:
# import math
# def milli_billi(n):
#     # get number of digits before the first number
#     digits = math.floor(math.log10(n)) 
    
#     if digits <= 5: #if < million, use thousands as unit
#         leading_n = round(n / 1e3, 2)
#         new_string = '$' + str(leading_n).replace('.0', '') + 'K'
#         return new_string
#     elif digits > 5 and digits <= 8: #if in the million range, use millions as units
#         leading_n = round(n / 1e6, 2)
#         new_string = '$' + str(leading_n).replace('.0', '') + ' Million'
#         return new_string
#     else: #if greater than millions, use billions
#         leading_n = round(n / 1e9, 2)
#         new_string = '$' + str(leading_n).replace('.0', '') + ' Billion'
#         return new_string

In [119]:
#make deal size in millions or billions
# final['deal_size_text'] = final.deal_size.apply(milli_billi, 1)

AttributeError: 'DataFrame' object has no attribute 'deal_size'

In [120]:
final.head()

,company,deal_text,publish_date,story_url,company_url
0,TTNews,"[ Lordstown Motors Corp., the electric-truck m...","[September 30, 2021]",/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/
1,TTNews,[ ABB Ltd. is preparing to roll out the first ...,"[September 30, 2021]",/articles/new-ev-charger-can-provide-62-miles-...,https://ttnews.com/
2,TTNews,[ After sounding an optimistic note in the fir...,"[September 30, 2021]",/articles/car-sales-drop-chip-shortages-stymie...,https://ttnews.com/
3,TTNews,[ WASHINGTON — His government overhaul plans a...,"[September 29, 2021]",/articles/progress-budget-talks-will-deal-beat...,https://ttnews.com/
4,TTNews,[ The outlook for transportation and logistics...,"[September 29, 2021]",/articles/what-q4-and-beyond-have-store-transp...,https://ttnews.com/


In [121]:
len(final)

25

In [123]:
#check previous posts, check to see there are no duplicates
# old = pd.read_csv('historical_deals.csv')
# dupes = pd.merge(final, old[['company', 'deal_size']], how = 'inner', on = ['company', 'deal_size'])
# final = final[~final.company.isin(dupes.company)]

EmptyDataError: No columns to parse from file

In [124]:
len(final)

25

In [125]:
final = final[final.company != 'Bolt']

In [126]:
final

,company,deal_text,publish_date,story_url,company_url
0,TTNews,"[ Lordstown Motors Corp., the electric-truck m...","[September 30, 2021]",/articles/lordstown-nears-deal-sell-ohio-plant...,https://ttnews.com/
1,TTNews,[ ABB Ltd. is preparing to roll out the first ...,"[September 30, 2021]",/articles/new-ev-charger-can-provide-62-miles-...,https://ttnews.com/
2,TTNews,[ After sounding an optimistic note in the fir...,"[September 30, 2021]",/articles/car-sales-drop-chip-shortages-stymie...,https://ttnews.com/
3,TTNews,[ WASHINGTON — His government overhaul plans a...,"[September 29, 2021]",/articles/progress-budget-talks-will-deal-beat...,https://ttnews.com/
4,TTNews,[ The outlook for transportation and logistics...,"[September 29, 2021]",/articles/what-q4-and-beyond-have-store-transp...,https://ttnews.com/
5,TTNews,[ LONDON — The British government’s reserve ta...,"[September 29, 2021]",/articles/soldiers-will-haul-fuel-ease-uk-crisis,https://ttnews.com/
6,TTNews,[ Navistar Inc. announced the launch of the re...,"[September 29, 2021]",/articles/navistar-launches-diesel-version-lat...,https://ttnews.com/
7,TTNews,[ Legal experts and researchers say juries gen...,"[September 29, 2021]",/articles/negative-jury-attitudes-can-contribu...,https://ttnews.com/
8,TTNews,[ Walmart Inc. is expanding its hiring push wi...,"[September 29, 2021]",/articles/walmart-stokes-hiring-rush-plan-add-...,https://ttnews.com/
9,TTNews,"[ When the first snub-nosed, electric van roll...","[September 29, 2021]",/articles/electric-vans-roll-line-once-made-ga...,https://ttnews.com/


In [127]:
#save deal records
# new_old = pd.concat([final, old], axis = 0)
new_old = pd.concat([final], axis = 0)

new_old.to_csv('historical_deals.csv', index = False)

NameError: name 'old' is not defined

In [ ]:
len(new_old)

2019

In [ ]:
date = 'Sunday, 08/29/2021'

title = '''
<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>{date}</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>
'''.format(date = date).strip()

In [ ]:
print(title)
pyperclip.copy(title)

<h1>
  Funded & Hiring
</h1>
<h2 style="text-align:center">
  <strong>Sunday, 08/29/2021</strong>
</h2>
<h2>
  A weekly roundup of funded startups and the jobs they're hiring for.
</h2>
<h4>
  Forwarded from a friend? 
  <a href = 'https://fundedandhiring.com/'>Subscribe</a> to stay up to date on the latest startup funding and job alerts.
</h4>


In [ ]:
content = ''''''

final_dict = final.to_dict(orient = 'records')
for row in final_dict:
    headline = "HEADLINE"
    company_name = row['company']
    story_url = row['story_url']
    description = row['deal_text']
    company_url = row['company_url']
    
    #get job types and job count
    # job_types = [key for key in row.keys() if key not in ['company', 'deal_text', 'publish_date', 'story_url', 'deal_size', 'job_page', 'deal_size_text', 'company_url']]
    # job_types_dict = {}
    # for typ in job_types:
    #     if row[typ] == 0:
    #         pass
    #     else:
    #         job_types_dict[typ] = row[typ]
            
    # job_types_dict = dict(sorted(job_types_dict.items(), key=lambda x: x[1], reverse=True))
    
    #order job types by number of jobs, move Other to the end
    # job_type_order = list(job_types_dict.keys())
    # if 'Other' in job_type_order:
    #     job_type_order.append('Other')
    #     del job_type_order[job_type_order.index('Other')]
    
    hiring_for = ''''''
    # for key in job_type_order:
    #     job_string = key + '({count})'.format(count = job_types_dict[key]) + ', '
    #     hiring_for += job_string
    # hiring_for = re.sub(r', $', '', hiring_for)
            
    
    entry = '''
    <tr>
      <td>
        <h3>
          <a href = "{story_url}">{headline}</a>
        </h3>
        <h4>
          <a href = "{company_url}">{company_name}</a>
        <strong>Funding Amount</strong>: <em><strong><a href = "{story_url}">{deal_size_text}</a></strong></em>
        <p>
          {description}
        </p>
        <p>
          <strong>Hiring For:</strong>
            <a href = "www.google.com">
              {hiring_for}
            </a>
        </p>
      </td>
    </tr> '''.format(story_url = story_url, company_name = company_name, deal_size_text = deal_size_text,deal_text = deal_text, job_page = 'www.google.com', hiring_for = hiring_for, company_url = company_url).strip()

    content += '\n' + entry.strip()

In [ ]:
pyperclip.copy(content)
print(content)


<tr>
      <td>
        <h3>
          <a href = "http://www.graymatter-robotics.com/">GrayMatter Robotics</a>
        </h3>
        <strong>Funding Amount</strong>: <em><strong><a href = "http://www.graymatter-robotics.com/">$4.1 Million</a></strong></em>
        <p>
          GrayMatter Robotics, an LA-based maker of manufacturing robots, raised 4.1 million in seed funding. Stage Venture Partners and Calibrate Ventures co-led, and were joined by 3M Ventures, OCA Ventures, Pathbreaker Ventures and B Capital Group.
        </p>
        <p>
          <strong>Hiring For:</strong>
            <a href = "www.google.com">
              
            </a>
        </p>
      </td>
    </tr>
<tr>
      <td>
        <h3>
          <a href = "https://www.runx.dev/">RunX</a>
        </h3>
        <strong>Funding Amount</strong>: <em><strong><a href = "http://axios.link/DRIR">$4.1 Million</a></strong></em>
        <p>
          RunX, an SF-based cloud infrastructure deployment startup, raised 4.1 

In [ ]:
#job boards
# stackoverflow, jobvite, bamboohr